Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [42]:
batch_size = 128
l2_reg_coeff = 0.01

lreg_graph = tf.Graph()

with lreg_graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                      shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, w) + b
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) \
        + l2_reg_coeff * (tf.nn.l2_loss(w) + tf.nn.l2_loss(b))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, w) + b)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, w) + b)

run_model(lreg_graph)

Initialized
Minibatch loss at step 0: 49.577827
Minibatch accuracy: 10.2%
Validation accuracy: 10.3%
Minibatch loss at step 500: 0.768924
Minibatch accuracy: 86.7%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 0.801391
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 0.570084
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.661553
Minibatch accuracy: 88.3%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.805596
Minibatch accuracy: 78.1%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.793747
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Test accuracy: 88.5%


In [98]:
def run_nn(batch_size=128, l2_reg_coeff=0.001, num_hidden_layers=1024, 
           dropout_keep_prob=1, num_using_train_data=-1, num_steps=3001):

    graph = tf.Graph()

    with graph.as_default():
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_layers]))
      biases_1 = tf.Variable(tf.zeros([num_hidden_layers]))
      weights_2 = tf.Variable(
        tf.truncated_normal([num_hidden_layers, num_labels]))
      biases_2 = tf.Variable(tf.zeros([num_labels]))

      layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
      layer_1 = tf.nn.dropout(layer_1, keep_prob=dropout_keep_prob)
      logits = tf.matmul(layer_1, weights_2) + biases_2

      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + l2_reg_coeff * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1)
                          + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))

      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
      valid_logits = tf.matmul(valid_layer_1, weights_2) + biases_2
      valid_prediction = tf.nn.softmax(valid_logits)

      test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
      test_logits = tf.matmul(test_layer_1, weights_2) + biases_2
      test_prediction = tf.nn.softmax(test_logits)
    
      with tf.Session(graph=graph) as session:
          tf.global_variables_initializer().run()
          print("Initialized")

          if num_using_train_data == -1:
            num_using_train_data = train_labels.shape[0]

          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (num_using_train_data - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
          print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


In [104]:
graph = run_nn(num_steps=5001)

Initialized
Minibatch loss at step 0: 651.147583
Minibatch accuracy: 7.8%
Validation accuracy: 28.8%
Minibatch loss at step 500: 197.727097
Minibatch accuracy: 85.9%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 115.295563
Minibatch accuracy: 82.8%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 68.751579
Minibatch accuracy: 89.1%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 41.589798
Minibatch accuracy: 88.3%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.395212
Minibatch accuracy: 87.5%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 15.570297
Minibatch accuracy: 85.2%
Validation accuracy: 86.2%
Minibatch loss at step 3500: 9.618038
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 5.950809
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 4500: 3.824389
Minibatch accuracy: 86.7%
Validation accuracy: 86.7%
Minibatch loss at step 5000: 2.529975
Minibatch accuracy: 88.3%
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [100]:
run_nn(num_using_train_data=batch_size*3)

Initialized
Minibatch loss at step 0: 590.342041
Minibatch accuracy: 11.7%
Validation accuracy: 28.9%
Minibatch loss at step 500: 190.406479
Minibatch accuracy: 100.0%
Validation accuracy: 72.6%
Minibatch loss at step 1000: 115.473068
Minibatch accuracy: 100.0%
Validation accuracy: 72.6%
Minibatch loss at step 1500: 70.029167
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 2000: 42.469486
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 2500: 25.755844
Minibatch accuracy: 100.0%
Validation accuracy: 72.6%
Minibatch loss at step 3000: 15.619955
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Test accuracy: 79.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [103]:
run_nn(dropout_keep_prob=0.5, num_steps=5001)

Initialized
Minibatch loss at step 0: 852.488770
Minibatch accuracy: 5.5%
Validation accuracy: 30.9%
Minibatch loss at step 500: 210.860748
Minibatch accuracy: 70.3%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 122.211769
Minibatch accuracy: 71.9%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 71.886749
Minibatch accuracy: 84.4%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 42.588230
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 25.934780
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 15.888635
Minibatch accuracy: 82.8%
Validation accuracy: 84.4%
Minibatch loss at step 3500: 9.860909
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 6.283962
Minibatch accuracy: 80.5%
Validation accuracy: 86.1%
Minibatch loss at step 4500: 3.889273
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 2.577422
Minibatch accuracy: 89.1%
Validatio

In [101]:
run_nn(dropout_keep_prob=0.5, num_using_train_data=batch_size*3)

Initialized
Minibatch loss at step 0: 807.269775
Minibatch accuracy: 8.6%
Validation accuracy: 37.3%
Minibatch loss at step 500: 191.644928
Minibatch accuracy: 100.0%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 116.270187
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 70.521530
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 2000: 42.773342
Minibatch accuracy: 100.0%
Validation accuracy: 75.3%
Minibatch loss at step 2500: 25.942644
Minibatch accuracy: 100.0%
Validation accuracy: 75.3%
Minibatch loss at step 3000: 15.734535
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Test accuracy: 82.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
